## 导入工具包

In [1]:
from py2neo import Graph, Node
import pandas as pd
from loguru import logger
from tqdm import tqdm

## 导入数据集csv文件

In [2]:
df = pd.read_csv('medical_data_all.csv')

In [3]:
df.shape

(8808, 23)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8808 entries, 0 to 8807
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   疾病名称    8808 non-null   object
 1   疾病描述    8807 non-null   object
 2   疾病种类    8807 non-null   object
 3   科室      8807 non-null   object
 4   病因      8791 non-null   object
 5   症状      8765 non-null   object
 6   检查      8804 non-null   object
 7   并发症     8525 non-null   object
 8   花费      8797 non-null   object
 9   疗程      8796 non-null   object
 10  疗法      8800 non-null   object
 11  治愈率     8757 non-null   object
 12  易感人群    8804 non-null   object
 13  感染概率    8797 non-null   object
 14  感染途径    8804 non-null   object
 15  预防措施    8771 non-null   object
 16  推荐药物    7645 non-null   object
 17  常用药物    7645 non-null   object
 18  具体药物    7645 non-null   object
 19  可以吃     5576 non-null   object
 20  不可以吃    5576 non-null   object
 21  推荐吃     5576 non-null   object
 22  是否纳入医保  8807 non-null   

In [5]:
df.describe()

,疾病名称,疾病描述,疾病种类,科室,病因,症状,检查,并发症,花费,疗程,...,感染概率,感染途径,预防措施,推荐药物,常用药物,具体药物,可以吃,不可以吃,推荐吃,是否纳入医保
count,8808,8807,8807,8807,8791,8765,8804,8525,8797,8796,...,8797,8804,8771,7645,7645,7645,5576,5576,5576,8807
unique,8807,8795,48,48,8707,8663,7841,2449,1218,758,...,1737,283,8102,1794,1464,2220,2294,1789,5523,2
top,胎膜早破,副神经节起源的肿瘤均属于APUD系统肿瘤，分为两类即：有生物活性的肿瘤和无生物活性的肿瘤。无...,"疾病百科,皮肤性病科,皮肤科","皮肤性病科,皮肤科",(一)发病原因\n病因尚不明。\n(二)发病机制\n发病机制还不清楚。,结节,皮损,休克,根据不同医院，收费标准不一致，市三甲医院约（5000——10000元）,1-3个月,...,0.001%,无传染性,本病暂无有效预防措施，早发现早诊断是本病防治的关键。,"布美他尼片,十一味金色丸,注射用布美他尼,注射用呋塞米,盐酸利多卡因注射液,注射用硫代硫酸钠...","头孢氨苄胶囊,盐酸左氧氟沙星胶囊","桂林南药布美他尼片(布美他尼片),雄巴拉曲神水十一味金色丸(十一味金色丸),西藏甘露仁青芒觉...","鸭蛋,鸡蛋,鸡肉,腰果","啤酒,白酒,绿豆,黑豆","罗汉果金银菜柿饼汤,双菇番茄黄瓜紫菜汤,紫菜芙蓉汤,菠菜双蛋羹,青菜饭,什菜面,冬菇椰菜豆腐...",否
freq,2,2,646,646,32,17,47,169,1113,1282,...,1117,7897,241,71,125,71,147,96,14,8806


## 实体（节点）

### 实体：所有症状

In [6]:
symptoms = []
for symptom in df['症状']:
    if pd.isna(symptom):
        continue
    try:
        symptoms.extend(symptom.split(','))
    except Exception as e:
        logger.warning(f"e: symptom:{symptom}, type of symptom{type(symptom)}")
        symptoms.extend(str(symptom).split(','))
symptoms = set(symptoms)
logger.info(symptoms)
logger.info(len(symptoms))

2024-12-19 21:35:16.644 | INFO     | __main__:<module>:11 - {'器质性木僵', '烧伤后恶心与呕吐', '肝郁气滞', '眼肌瘫痪', '卵巢增大', '尿急', '呼吸道阻塞', '动脉血氧饱和度降低', '神经性头痛', '四肢血压脉搏不对称', '耳道流脓', '颈短', '钩回发作', '尿频', '肺纹理增粗', '肠道血管畸形', '胸口闪电疼痛', '含铁血黄素沉着症', '肉芽肿', '上行性皮肤麻木', '指甲突然变白', '角回受损', '尿酸盐在关节内...', '抗Jo-1抗体阳性', '中度昏迷', '牙列稀疏', '多疑', '骨髓损害', '过劳', '体重增加', '下体异味', '产后小腹坠痛', '骶尾部急性脓肿', '癌症晚期胃肠道症状', '异常体位所致的窒息', '耐热力降低', '胎盘植入', '骨软化', '脑发育缓慢', '胃扭转梗阻症状', '驻站医', '胸锁乳突肌附着处压痛', '肢端溃疡', '心脏失代偿', '面色苍白', '发炎', '术后发热', '图形定向障碍', '酒精中毒性幻觉症', '嗜红细胞综合征', '醉酒', '目赤', '掌跖角化过度', '迁徙性浅静脉血栓形成', '唇角脱屑', '腮腺无痛性肿块', '子宫增大明显（...', '胃石形成', '闫红梅', '耳垂粘连', '桡骨茎突剧痛', '恶息、多泡味的...', '绿牙或棕褐色牙', '脓疱性痤疮', '肢体远端出现腕垂踝垂', '心包破损伤', '咽喉炎', '颈部皱纹多', '子宫呈葫芦状', '痉挛性偏瘫步态', '脐周腹壁缺损', '胸闷气短', '体位性眩晕', '阴茎勃起不硬', '核黄疸', '肘部的肿胀压痛', '小儿疳积', '紧张', '眼球内陷', '虚汗', '舌头溃疡', '慢性贫血', '角膜有菌苔或溃疡', '粘液性水肿面容', '眼球挫伤', '一咳嗽胸口就疼', '脐带过短或相对过短', '舌乳头消失', '粪便硬结如羊粪状', '乳头内陷', '细胞色素C氧化...', '眩晕', '完全性遗忘症', '营养代谢缺乏', '胸痛伴胸闷、心悸', '胸椎疼痛', '舌乳头增生', '臂痛', '手指歪斜', '尿等待', '肠痉挛', 

In [7]:
len(symptoms)

5998

### 所有科室

In [8]:
departments = []
for department in df['科室']:
    if pd.isna(department):
        continue
    departments.extend(department.split(','))
departments = set(departments)
logger.info(departments)
logger.info(len(departments))

2024-12-19 21:35:18.299 | INFO     | __main__:<module>:7 - {'血液科', '肿瘤科', '遗传病科', '其他科室', '风湿免疫科', '肾内科', '心胸外科', '泌尿外科', '儿科综合', '肿瘤内科', '传染科', '消化内科', '泌尿内科', '中医综合', '耳鼻喉科', '呼吸内科', '营养科', '烧伤科', '感染科', '精神科', '产科', '妇产科', '中医科', '急诊科', '内分泌科', '肝病', '眼科', '康复科', '肝胆外科', '男科', '神经外科', '小儿外科', '神经内科', '皮肤性病科', '心理科', '骨外科', '普外科', '五官科', '外科', '肛肠科', '小儿内科', '心内科', '儿科', '不孕不育', '肿瘤外科', '其他综合', '皮肤科', '性病科', '减肥', '生殖健康', '口腔科', '内科', '妇科', '整形美容科'}
2024-12-19 21:35:18.301 | INFO     | __main__:<module>:8 - 54


### 实体：所有检查

In [9]:
checks = []
for check in df['检查']:
    if pd.isna(check):
        continue
    checks.extend(check.split(','))
checks = set(checks)
logger.info(checks)
logger.info(len(checks))

2024-12-19 21:35:19.618 | INFO     | __main__:<module>:7 - {'可提取性核抗原（ENA）', '高钠试验', '糖链抗原72-4', '尿肾上腺素', '过氯酸盐释放试验', '血浆游离血红蛋白', 'Kveim试验', '肛腹诊', '血浆渗透压（POP）', '抑癌基因Rb', '水溶性维生素', '放射免疫分析', '近红外线乳腺扫描检查', '酸化甘油溶血试验', '生育镜', '血红蛋白', '膀胱镜', '骨髓粒细胞系统', '肩外展摆动试验', '头颅正侧位片', '胸部外部检查', '甘油复红试验', '血细胞分析仪', '脏器断层显像', '脑脊液胆固醇酯', '血清肌酸激酶', '椭圆形红细胞', '新生儿红细胞计数', '动态心电图（Holter监测）', '造影检查', '总甲状腺素', '红细胞δ-氨基-γ-酮戊酸脱水酶', '抗丁型肝炎病毒IgM抗体', '精囊检查', '丙型肝炎病毒抗体免疫球蛋白G', 'CT平扫', '颅内多普勒血流图(TCD)', '乙肝两对半', '优生五项', '抗肝细胞膜抗原抗体(ALMA)', '血清淀粉样蛋白A', '钼靶X线检查', '肠鸣音', '泪液β2-微球蛋白', '尿去氢异雄酮', '谷氨酰胺', '胆道造影', '胰岛素胃酸分泌量', '异柠檬酸脱氢酶', '呼吸道粘液-纤毛清除功能测定', '压平眼压计测量法', '精液细胞', '血清纤维结合蛋白', '二氧化碳分压（PCO2', '肌红蛋白', '血清磷脂', '眼位检查法', 'Ag）', '免疫浊度技术', 'E-玫瑰花环形成试验', '释放抗体试验', '血清胆固醇与磷脂比值', '粪便性状', '皮肤斑贴实验', '抗角蛋白抗体(AKA)', '视神经孔片', '红细胞膜磷脂', '尿铁（Fe）', '咬合关系检查', '阴道活检', '滑液检查', '维生素D（VitD）', '乳房淋巴结实验', '游离三碘甲状腺原氨酸指数', '抗组织相容性抗原-DR抗体', '每分钟最大通气量（MVV）', 'D860激发试验', '端坐伸膝试验', '弓形虫点免疫金渗滤法', '钩端螺旋体抗体', '尿渗透压', '血清肌红蛋白', '血管壁的检测', '血吸虫皮内试验', '皮

### 实体：所有药物

In [10]:
drugs = []
for drug in df['推荐药物']:
    if pd.isna(drug):
        continue
    try:
        drugs.extend(drug.split(','))
    except Exception as e:
        # pass
        logger.info(e)
        
for drug_ in df['常用药物']:
    if pd.isna(drug_):
        continue
    try:
        drugs.extend(drug_.split(','))
    except Exception as e:
        logger.info(e)
        # pass
drugs = set(drugs)
logger.info(drugs)
logger.info(len(drugs))

2024-12-19 21:35:21.248 | INFO     | __main__:<module>:20 - {'盐酸吗啉胍片', '葡萄糖酸钙含片', '23', '瓜霜退热灵胶囊', '吡拉西坦葡萄糖注射液', '脂肪乳注射液[C14-24]', '碘甘油', '化痰平喘片', '铝碳酸镁片', '翁沥通颗粒', '香菇多糖片', '苯丙氨酯片', '阑尾灵颗粒', '成膜型', '金振口服液', '乳酸左氧氟沙星分散片', '六神胶囊', '加味藿香正气丸', '盐酸消旋山莨菪碱注射液', '阿法骨化醇片', '鞣酸蛋白酵母散', '吡拉西坦颗粒', '盐酸奥昔布宁缓释片', '复方阿胶补血膏', '丙硫氧嘧啶肠溶片', '生化丸', '生精片', '盐酸克林霉素棕榈酸酯分散片', '四妙丸', '胃蛋白酶颗粒', '益康胶囊', '十味诃子丸', '治咳枇杷露', '盐酸小檗碱片', '复方乌鳢口服液', '五维牛磺酸口服溶液', '三合钙咀嚼片', '复方托吡卡胺滴眼液', '阿奇霉素分散片', '障眼明片', '泮托拉唑钠肠溶片', '更昔洛韦胶囊', '经带宁胶囊', '地红霉素肠溶片', '林可霉素利多卡因凝胶', '蛇莲胶囊', '盐酸芦氟沙星胶囊', '消癌平胶囊', '鱼石脂软膏', '石淋通片', '妇科白带片', '巴氯芬片', '呋塞米注射液', '益气聪明丸', '肾肝宁胶囊', '注射用胸腺五肽', '硫酸锌口服溶液', '盐酸地芬尼多片', '花藤子颗粒', '阿莫西林克拉维酸钾(4:1)分散片', '别嘌醇片', '复方溴咖片', '牛黄抱龙丸', '宝宝乐', '卡托普利片', '三蛇胆川贝糖浆', '右旋糖酐羟丙甲纤维素滴眼液', '茴香橘核丸', '氟康唑分散片', '金骨莲片', '桂枝茯苓丸', '二母宁嗽丸', '盐酸氨溴索注射液', '阿莫西林克拉维酸钾干混悬剂', '琥乙红霉素咀嚼片', '注射用醋酸亮丙瑞林微球', '马来酸氟伏沙明片', '甲硝唑乳膏', '卡培他滨片', '注射用氢化可的松琥珀酸钠', '金胆片', '司莫司汀胶囊', '甲磺酸双氢麦角毒碱片', '癫痫宁片', '痔炎消颗粒', '肾复康片', '消糜栓', '盐酸丁螺环酮片', '脉络舒通颗粒', '加味逍遥丸', '红霉素软膏'

### 实体：所有食物

In [11]:
foods = []
for food in df['可以吃']:
    if pd.isna(food):
        continue
    try:
        foods.extend(food.split(','))
    except Exception as e:
        logger.warning(e)
        # pass
for food_ in df['不可以吃']:
    if pd.isna(food_):
        continue
    try:
        foods.extend(food_.split(','))
    except Exception as e:
        logger.warning(e)
        # pass
for each in df['推荐吃']:
    if pd.isna(each):
        continue
    try:
        foods.extend(each.split(','))
    except Exception as e:
        logger.warning(e)
        # pass
foods = set(foods)
logger.info(foods)
logger.info(len(foods))

2024-12-19 21:35:22.905 | INFO     | __main__:<module>:27 - {'蒲公英茶', '芝麻酱菜', '菠萝粥', '水晶莲子', '苋菜黄鱼羹', '黄连白头翁粥', '苜蓿', '草莓沙拉拌生菜', '参芪牛肉', '酱野猪肉', '鸭心燕窝汤', '人参茯苓汤', '豆腐炖菜心', '空心菜粥', '蛋糕', '芝麻鸭肉', '食盐炒芝麻', '番茄马铃薯汤', '归地烧羊肉', '薏仁粥', '双皮鲫鱼', '西兰花炒牛肉', '瓜粒杂锦汤', '萝卜粳米粥', '胡萝卜粥', '五香鲤鱼块', '驴肉水饺', '白菜清汤', '红枣炖猪肉', '酸菜豌豆汤', '小白菜芋头', '牛肉馅饼', '银鱼芥菜粥', '油菜薹', '芹菜拌腐竹', '鸳鸯河虾', '飘香四季豆', '双红南瓜汤', '老黄瓜瘦肉汤', '肉末南瓜', '萝卜虾汤', '黄芪粥', '蜜枣桂圆粥', '蛋白芹菜', '菊芋粥', '荠菜包', '大肠扒油菜', '什锦炒木耳', '杏鲍菇猪肉汤', '玫瑰球拼芝麻鱼条', '菠菜腰花', '天麻鱼头粥', '西芹芦笋贝', '鸡蛋酿苹果', '荠菜炒鸡丁', '干炸带鱼', '萝卜粥', '黑芝麻果仁粥', '莲子鸡丁', '紫苏叶汁', '红萝卜瘦肉鸡肝笋菇汤', '虾皮菠菜', '冬笋拌油菜', '白扁豆粥', '豆腐皮粥', '奶香菜花', '虎皮鸽蛋', '脆爆海带', '白菜藕丸汤', '肉丝炒饼', '香干拌马兰头', '茄汁烩肉饭', '乳鸽银耳汤', '油菜松', '糖醋青萝卜丝', '天麻鱼片', '卤鸭翅膀', '芦笋粥', '罗汉果', '猪心莲子汤', '皮蛋芦笋汤', '桃仁丝瓜', '烧肉蛋包饭', '鱼蛋豆芽汤', '虾肉馄饨', '番茄洋葱芹菜汁', '德式土豆饼', '青鸭羹', '冰花鸡蛋球', '参枣炖蘑菇', '猪头肉', '莴笋', '苹果汁', '芝麻栗子糕', '桑葚饮', '淡菜汤', '海参汤', '紫菜豆腐虾米瘦肉汤', '板栗烧菜心', '枇杷银耳汤', '酱苹果', '核桃牛奶饮', '清莲花豆腐', '脆皮茄子', '鱿鱼蛤蜊粥', '绿豆', '蒜茸野菊花', '绿豆马齿苋汤', '白菜海带丝', '牛肉菜粒羹', '莲

### 实体：所有药物厂商

In [12]:
producers = []

for each in df['具体药物']:
    if pd.isna(each):
        continue
    try:
        for each_drug in each.split(','):
            producer = each_drug.split('(')[0]
            producers.append(producer)
    except Exception as e:
        logger.warning(e)
        # pass
producers = set(producers)
logger.info(producers)
logger.info(len(producers))

2024-12-19 21:35:24.322 | INFO     | __main__:<module>:14 - {'陕西步长中风回春胶囊', '天丁硫酸镁注射液', '海南葫芦娃肠炎宁颗粒', '浙江南洋药业阿奇霉素胶囊', '万高药业羟苯磺酸钙分散片', '河南同源氧氟沙星滴耳液', '辽宁富东阿奇霉素分散片', '安庆药业补中益气丸', '太原制药脑络通胶囊', '同仁堂制药厂五子衍宗丸', '沈阳兴齐眼药牛磺酸滴眼液', '永寿制药葡萄糖注射液', '山东华鲁酚磺乙胺氯化钠注射', '哈药三精千鹤根痛平胶囊', '中和制药牛磺酸颗粒', '扬州艾迪制药琥乙红霉素片', '皇甫谧制药止血宝片', '福瑞达制药颈痛片', '亚东生物血尿胶囊', '黑龙江百泰对乙酰氨基酚片', '童奥特地牡宁神口服液', '太平风油精', '陕西医药参芪升阳补血胶囊', '成都第一复方四环素泼尼松膜', '寿堂盐酸美西律片', '卫生材料药业水杨酸苯酚贴膏', '川抗环孢素口服溶液', '莘原制药羚羊感冒片', '山东华信制药碳酸氢钠注射液', '浙江安贝特小儿硫酸亚铁糖浆', '百会药业菠萝蛋白酶肠溶片', '九泰药业奥美拉唑肠溶胶囊', '力强药业小儿暖脐膏', '润弘制药肝胆舒康胶囊', '神华维康天麻蜜环菌片', '三联药业注射用腺苷钴铵', '康乐果肾炎四味胶囊', '哈药三精儿童赖氨酸磷酸氢钙', '辽宁金丹药业加味逍遥丸', '御大夫通脉颗粒', '欧化药业盐酸洛美沙星片', '昕维甲磺酸伊马替尼片', '长春海悦胸腺肽注射液', '开封制药消旋山莨菪碱片', '华润双鹤药业注射用硫普罗宁', '精优药业转移因子口服溶液', '中狮厄贝沙坦分散片', '宜昌人福盐酸金刚烷胺片', '昊畅', '漯河南街村全威制药吡拉西坦', '禾丰乙酰谷酰胺注射液', '注射用胸腺五肽', '康美保宁心脑欣丸', '青阳盐酸左旋咪唑片', '宁波大红鹰盐酸胺碘酮片', '康博士制药一清片', '力源制药抑亢散', '山东凤凰环磷腺苷葡胺注射液', '广东三蓝天王补心丸', '吉林恒和维康眼氨肽注射液', '立健新', '润众恩替卡韦分散片', '南岳生物制药狂犬病人免疫球', '庆安药业维生素E软胶囊', '华青曲咪新乳膏', '银涛药业保胎灵胶囊', '重庆希尔安蒲公英颗粒

### 疾病字典信息

In [13]:
disease_infos = [] # 疾病信息
for idx, row in df.iterrows():
    disease_infos.append(dict(row))
logger.info(disease_infos[:6])
logger.info(len(disease_infos))

2024-12-19 21:35:27.104 | INFO     | __main__:<module>:4 - [{'疾病名称': '肺泡蛋白质沉积症', '疾病描述': '肺泡蛋白质沉积症(简称PAP)，又称Rosen-Castle-man-Liebow综合征，是一种罕见疾病。该病以肺泡和细支气管腔内充满PAS染色阳性，来自肺的富磷脂蛋白质物质为其特征，好发于青中年，男性发病约3倍于女性。', '疾病种类': '疾病百科,内科,呼吸内科', '科室': '内科,呼吸内科', '病因': '病因未明，推测与几方面因素有关：如大量粉尘吸入（铝，二氧化硅等），机体免疫功能下降（尤其婴幼儿），遗传因素，酗酒，微生物感染等，而对于感染，有时很难确认是原发致病因素还是继发于肺泡蛋白沉着症，例如巨细胞病毒，卡氏肺孢子虫，组织胞浆菌感染等均发现有肺泡内高蛋白沉着。\n虽然启动因素尚不明确，但基本上同意发病过程为脂质代谢障碍所致，即由于机体内，外因素作用引起肺泡表面活性物质的代谢异常，到目前为止，研究较多的有肺泡巨噬细胞活力，动物实验证明巨噬细胞吞噬粉尘后其活力明显下降，而病员灌洗液中的巨噬细胞内颗粒可使正常细胞活力下降，经支气管肺泡灌洗治疗后，其肺泡巨噬细胞活力可上升，而研究未发现Ⅱ型细胞生成蛋白增加，全身脂代谢也无异常，因此目前一般认为本病与清除能力下降有关。', '症状': '紫绀,胸痛,呼吸困难,乏力,毓卓', '检查': '胸部CT检查,肺活检,支气管镜检查', '并发症': '多重肺部感染', '花费': '根据不同医院，收费标准不一致，省市三甲医院约（ 8000——15000 元）', '疗程': '约3个月', '疗法': '支气管肺泡灌洗', '治愈率': '约40%', '易感人群': nan, '感染概率': '0.00002%', '感染途径': '无传染性', '预防措施': '1、避免感染分支杆菌病，卡氏肺囊肿肺炎，巨细胞病毒等。\n2、注意锻炼身体，提高免疫力。', '推荐药物': nan, '常用药物': nan, '具体药物': nan, '可以吃': nan, '不可以吃': nan, '推荐吃': nan, '是否纳入医保': '否'}, {'疾病名称': '百日咳', '疾病描述': '百日咳(pertussis，whoopingcou

In [14]:
dict(row).keys()

dict_keys(['疾病名称', '疾病描述', '疾病种类', '科室', '病因', '症状', '检查', '并发症', '花费', '疗程', '疗法', '治愈率', '易感人群', '感染概率', '感染途径', '预防措施', '推荐药物', '常用药物', '具体药物', '可以吃', '不可以吃', '推荐吃', '是否纳入医保'])

## 关系（边）

In [15]:
def deduplicate(rels_old):
    '''关系去重函数'''
    rels_new = []
    for each in rels_old:
        if each not in rels_new:
            rels_new.append(each)
    return rels_new

### 关系：疾病-检查

In [16]:
rels_check = []
for idx, row in df.iterrows():
    if pd.isna(row['检查']):
        continue
    for each in row['检查'].split(','):
        rels_check.append([row['疾病名称'], each])
rels_check = deduplicate(rels_check)

In [17]:
rels_check[:6]

[['肺泡蛋白质沉积症', '胸部CT检查'],
 ['肺泡蛋白质沉积症', '肺活检'],
 ['肺泡蛋白质沉积症', '支气管镜检查'],
 ['百日咳', '耳、鼻、咽拭子细菌培养'],
 ['百日咳', '周围血白细胞计数及分类检验'],
 ['百日咳', '血常规']]

In [18]:
len(rels_check)

39594

### 关系：疾病-症状

In [19]:
rels_symptom = []
for idx, row in df.iterrows():
    if pd.isna(row['症状']):
        continue
    for each in row['症状'].split(','):
        rels_symptom.append([row['疾病名称'], each])
rels_symptom = deduplicate(rels_symptom)

In [20]:
rels_symptom[:6]

[['肺泡蛋白质沉积症', '紫绀'],
 ['肺泡蛋白质沉积症', '胸痛'],
 ['肺泡蛋白质沉积症', '呼吸困难'],
 ['肺泡蛋白质沉积症', '乏力'],
 ['肺泡蛋白质沉积症', '毓卓'],
 ['百日咳', '吸气时有蝉鸣音']]

In [21]:
len(rels_symptom)

54710

### 关系：疾病-疾病（并发症）

In [22]:
rels_acompany = []
for idx, row in df.iterrows():
    if pd.isna(row['并发症']):
        continue
    for each in row['并发症'].split(','):
        rels_acompany.append([row['疾病名称'], each])
rels_acompany = deduplicate(rels_acompany)

In [23]:
rels_acompany[:6]

[['肺泡蛋白质沉积症', '多重肺部感染'],
 ['百日咳', '肺不张'],
 ['苯中毒', '贫血'],
 ['喘息样支气管炎', '支气管哮喘'],
 ['成人呼吸窘迫综合征', '细菌性肺炎'],
 ['大量羊水吸入', '呼吸衰竭']]

In [24]:
len(rels_acompany)

12024

### 关系：疾病-推荐药物

In [25]:
rels_recommanddrug = []
for idx, row in df.iterrows():
    if pd.isna(row['推荐药物']):
        continue
    try:
        for each in row['推荐药物'].split(','):
            rels_recommanddrug.append([row['疾病名称'], each])
    except:
        pass
rels_recommanddrug = deduplicate(rels_recommanddrug)

### 关系：疾病-常用药物

In [26]:
rels_common_drug = []
for idx, row in df.iterrows():
    if pd.isna(row['常用药物']):
        continue
    try:
        for each in row['常用药物'].split(','):
            rels_common_drug.append([row['疾病名称'], each])
    except:
        pass
rels_common_drug = deduplicate(rels_common_drug)

### 关系：疾病-不可以吃

In [27]:
rels_noteat = []
for idx, row in df.iterrows():
    if pd.isna(row['不可以吃']):
        continue
    try:
        for each in row['不可以吃'].split(','):
            rels_noteat.append([row['疾病名称'], each])
    except:
        pass
rels_noteat = deduplicate(rels_noteat)

### 关系：疾病-可以吃

In [28]:
rels_doeat = []
for idx, row in df.iterrows():
    if pd.isna(row['可以吃']):
        continue
    try:
        for each in row['可以吃'].split(','):
            rels_doeat.append([row['疾病名称'], each])
    except:
        pass
rels_doeat = deduplicate(rels_doeat)

### 关系：疾病-推荐吃

In [29]:
rels_recommandeat = []
for idx, row in df.iterrows():
    if pd.isna(row['推荐吃']):
        continue
    try:
        for each in row['推荐吃'].split(','):
            rels_recommandeat.append([row['疾病名称'], each])
    except:
        pass
rels_recommandeat = deduplicate(rels_recommandeat)

### 关系：药物厂商-具体药物

In [30]:
rels_drug_producer = []
for each in df['具体药物']:
    if pd.isna(each):
        continue
    try:
        for each_drug in each.split(','):
            if pd.isna(each_drug):
                continue
            producer = each_drug.split('(')[0]
            drug = each_drug.split('(')[1][:-1]
            rels_drug_producer.append([producer, drug])
    except:
        pass
rels_drug_producer = deduplicate(rels_drug_producer)

### 关系：疾病-科室、小科室-大科室

In [31]:
rels_category = [] # 关系：疾病-科室
rels_department = [] # 关系：小科室-大科室
for idx, row in df.iterrows():
    if pd.isna(row['科室']):
        continue
    if len(row['科室'].split(',')) == 1:
        rels_category.append([row['疾病名称'], row['科室']])
    else:
        big = row['科室'].split(',')[0] # 大科室
        small = row['科室'].split(',')[1] # 小科室
        rels_category.append([row['疾病名称'], small])
        rels_department.append([small, big])
rels_category = deduplicate(rels_category)
rels_department = deduplicate(rels_department)

In [32]:
rels_category[:6]

[['肺泡蛋白质沉积症', '呼吸内科'],
 ['百日咳', '小儿内科'],
 ['苯中毒', '急诊科'],
 ['喘息样支气管炎', '呼吸内科'],
 ['成人呼吸窘迫综合征', '呼吸内科'],
 ['大量羊水吸入', '小儿内科']]

In [33]:
rels_department[:6]

[['呼吸内科', '内科'],
 ['小儿内科', '儿科'],
 ['其他综合', '其他科室'],
 ['肿瘤内科', '肿瘤科'],
 ['心胸外科', '外科'],
 ['感染科', '外科']]

## 连接图数据库

In [34]:
# 注意，这里的用户名为neo4j全局用户名，而非DBMS或者database的名称
g = Graph(auth=('neo4j', 'neo4jneo4j'))

## 创建知识图谱实体（节点）

### 样例代码

In [35]:
# # Neo4j样例代码

# 创建实体
# node = Node('Disease', name='百日咳', easy_get='多见于小儿')
# g.create(node)

In [36]:
# 删除所有实体和关系
cypher = 'MATCH (n) DETACH DELETE n'
g.run(cypher)

(No data)

### 创建疾病实体

In [37]:
count = 0
for disease_dict in tqdm(disease_infos):
    try:
        node = Node("Disease",
                    name=disease_dict['疾病名称'],
                    desc=disease_dict['疾病描述'],
                    prevent=disease_dict['预防措施'],
                    cause=disease_dict['病因'],
                    easy_get=disease_dict['易感人群'],
                    cure_lasttime=disease_dict['疗程'],
                    cure_department=disease_dict['科室'],
                    cure_way=disease_dict['疗法'], 
                    cured_prob=disease_dict['治愈率']
                    )
        g.create(node)
        count += 1
        # logger.info(f"创建疾病实体：{disease_dict['疾病名称']}")
    except Exception as e:
        logger.warning(e)
logger.info(f'共创建 {count} 个疾病实体')

100%|██████████| 8808/8808 [01:23<00:00, 104.86it/s]
2024-12-19 21:41:23.255 | INFO     | __main__:<module>:20 - 共创建 8808 个疾病实体


### 创建药物实体(只有name)

In [38]:
count = 0
for each in tqdm(drugs):
    node = Node('Drug', name=each)
    g.create(node)
    # logger.info(f'创建实体 {each}')
    count += 1
logger.info(f"创建{count}个Drug实体")

100%|██████████| 3829/3829 [00:25<00:00, 149.84it/s]
2024-12-19 21:42:06.047 | INFO     | __main__:<module>:7 - 创建3829个Drug实体


### 创建食物实体(只有name)

In [39]:
count = 0
for each in tqdm(foods):
    node = Node('Food', name=each)
    g.create(node)
    # logger.info(f'创建实体 {each}')
    count += 1
logger.info(f"创建{count}个Food实体")

100%|██████████| 4870/4870 [00:35<00:00, 136.48it/s]
2024-12-19 21:42:41.750 | INFO     | __main__:<module>:7 - 创建4870个Food实体


### 创建检查实体(只有name)

In [40]:
count = 0
for each in tqdm(checks):
    node = Node('Check', name=each)
    g.create(node)
    count += 1
logger.info(f"创建{count}个Check实体")

100%|██████████| 3397/3397 [00:25<00:00, 134.78it/s]
2024-12-19 21:43:06.965 | INFO     | __main__:<module>:6 - 创建3397个Check实体


### 创建科室实体(只有name)

In [41]:
count = 0
for each in tqdm(departments):
    node = Node('Department', name=each)
    g.create(node)
    count += 1
logger.info(f"创建{count}个Department实体")

100%|██████████| 54/54 [00:00<00:00, 137.38it/s]
2024-12-19 21:43:38.687 | INFO     | __main__:<module>:6 - 创建54个Department实体


### 创建 药物厂商 实体(只有name)

In [42]:
count = 0
for each in tqdm(producers):
    node = Node('Producer', name=each)
    g.create(node)
    count += 1
logger.info(f"创建{count}个Producer实体")

100%|██████████| 17202/17202 [02:02<00:00, 140.90it/s]
2024-12-19 21:45:43.913 | INFO     | __main__:<module>:6 - 创建17202个Producer实体


### 创建 症状 实体(只有name)

In [43]:
count = 0
for each in tqdm(symptoms):
    node = Node('Symptom', name=each)
    g.create(node)
    count += 1
logger.info(f"创建{count}个Symptom实体")

100%|██████████| 5998/5998 [00:44<00:00, 134.77it/s]
2024-12-19 21:46:28.440 | INFO     | __main__:<module>:6 - 创建5998个Symptom实体


## 创建知识图谱关系（连接、边）

### 样例代码

In [44]:
# # Neo4j样例代码

# # 删除所有实体和关系
# cypher = 'MATCH (n) DETACH DELETE n'
# g.run(cypher)

# # 删除 关系
# cypher_ = 'MATCH p=()-[r]->() DELETE r' # 'MATCH p=()-[r:need_check]->() DELETE r' # 删除指定的need_check关系
# g.run(cypher_)

In [45]:
# # 创建关系 样例代码
# start_node = 'Disease'
# end_node = 'Check'
# p = '百日咳'
# q = '血常规'
# rel_type = 'need_check'
# rel_name = '诊断检查'

# # Cypher语句
# query = "match(p:%s),(q:%s) where p.name='%s' and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (start_node, end_node, p, q, rel_type, rel_name)
# print(query)
# g.run(query) # 运行 Cypher 语句


In [46]:
def create_relationship(start_node, end_node, edges, rel_type, rel_name):
    count = 0
    p_set = set()
    q_set = set()
    '''创建关系函数'''
    for edge in tqdm(edges):
        p = edge[0]
        q = edge[1]
        p_set.add(p)
        q_set.add(q)
        # 创建关系的 Cypher 语句
        query = """match(p:%s),(q:%s) where p.name="%s" and q.name="%s" create (p)-[rel:%s{name:"%s"}]->(q)""" % (start_node, end_node, p, q, rel_type, rel_name)
        try:
            g.run(query) # 运行 Cypher 语句
            count += 1
            # logger.info(f'创建关系 {p}-{rel_type}->{q}')
        except Exception as e:
            logger.warning(e)
    logger.info(f"共创建{count}组{start_node}->{end_node}的 {rel_name} 关系, 涉及{len(p_set)}个{start_node}节点和{len(q_set)}个{end_node}节点")

### 创建所有关系

In [47]:
rels_check[:6]

[['肺泡蛋白质沉积症', '胸部CT检查'],
 ['肺泡蛋白质沉积症', '肺活检'],
 ['肺泡蛋白质沉积症', '支气管镜检查'],
 ['百日咳', '耳、鼻、咽拭子细菌培养'],
 ['百日咳', '周围血白细胞计数及分类检验'],
 ['百日咳', '血常规']]

In [48]:
create_relationship('Disease', 'Check', rels_check, 'need_check', '诊断检查')
create_relationship('Disease', 'Symptom', rels_symptom, 'has_symptom', '症状')
create_relationship('Disease', 'Disease', rels_acompany, 'acompany_with', '并发症')
create_relationship('Disease', 'Department', rels_category, 'belongs_to', '所属科室')
create_relationship('Disease', 'Food', rels_recommandeat, 'recommand_eat', '推荐食谱')
create_relationship('Disease', 'Food', rels_noteat, 'no_eat', '忌吃')
create_relationship('Disease', 'Food', rels_doeat, 'do_eat', '宜吃')
create_relationship('Department', 'Department', rels_department, 'belongs_to', '属于')
create_relationship('Disease', 'Drug', rels_common_drug, 'common_drug', '常用药品')
create_relationship('Producer', 'Drug', rels_drug_producer, 'drugs_of', '生产药品')
create_relationship('Disease', 'Drug', rels_recommanddrug, 'recommand_drug', '好评药品')

100%|██████████| 39594/39594 [13:34<00:00, 48.60it/s]
2024-12-19 22:00:26.058 | INFO     | __main__:create_relationship:19 - 共创建39594组Disease->Check的 诊断检查 关系, 涉及8803个Disease节点和3397个Check节点
100%|██████████| 54710/54710 [20:06<00:00, 45.35it/s]
2024-12-19 22:20:32.524 | INFO     | __main__:create_relationship:19 - 共创建54710组Disease->Symptom的 症状 关系, 涉及8764个Disease节点和5998个Symptom节点
100%|██████████| 12024/12024 [05:05<00:00, 39.30it/s]
2024-12-19 22:25:38.506 | INFO     | __main__:create_relationship:19 - 共创建12024组Disease->Disease的 并发症 关系, 涉及8524个Disease节点和2205个Disease节点
100%|██████████| 8806/8806 [03:00<00:00, 48.73it/s]
2024-12-19 22:28:39.226 | INFO     | __main__:create_relationship:19 - 共创建8806组Disease->Department的 所属科室 关系, 涉及8806个Disease节点和48个Department节点
100%|██████████| 40221/40221 [12:35<00:00, 53.25it/s]
2024-12-19 22:41:14.489 | INFO     | __main__:create_relationship:19 - 共创建40221组Disease->Food的 推荐食谱 关系, 涉及5575个Disease节点和4506个Food节点
100%|██████████| 22239/22239 [06:35<00:00, 56.2

![jupyter](./neo4j_all_nodes_rels.png)